In [11]:
# Import dependencies
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from utils_common import get_db_engine, encode_dataframe

import warnings
warnings.filterwarnings('ignore')

# **************************************
# keras.models.Sequential()
# **************************************
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint # keras, not kera


In [12]:
# **************************************
# Neural Net Configuration Variables
# **************************************

# ******************
# Data Configuration
tablename = 'people_vet_join'
y_column = 'TotalVets'

# ******************
# First Layer Configuration
kS_nn_first_units       = 8
kS_nn_first_activation  = "relu"

# ******************
# Output Layer Configuration
kS_nn_output_units      = 1
kS_nn_output_activation = "sigmoid"

# ******************
# Deep Learning Configuration
kS_nn_deep_layers       = [
    # Comment out the next two lines to disable "deep learning"
    {"units": 16,
     "activation": "relu"},

    # Add more layers with
    #    {"units": <units>,
    #     "activation": <activation>},
                     ]

# ******************
# Compiler Configuration
kS_nn_compile_loss      = "binary_crossentropy"
kS_nn_compile_optimizer = "adam"
kS_nn_compile_metrics   = ["accuracy"]

# ******************
# Training Configuration
kS_nn_train_epochs      = 100

# ******************
# Output File
kS_nn_file              = "kS-Enlistment_Prediction.h5"

In [13]:
# Import and read data
# combined_df = getfromdatabase{postres}

try:
    db_engine = get_db_engine()
except Exception as e:
    print(f"\nFailed to connect to database engine.\n", e)

try:
    combined_df = pd.read_sql_table(tablename, db_engine)
except Exception as e:
    print(f"\nFailed to read table {tablename} on db_engine {db_engine}.\n", e)
   

In [14]:
df_labels = combined_df[['State', 'County']]
combined_df = combined_df.drop(columns=['FIPS', 'State', 'County'])


In [15]:
combined_df.head()

,PopChangeRate1819,PopChangeRate1019,TotalPopEst2019,NetMigrationRate1019,NaturalChangeRate1019,Net_International_Migration_Rate_2010_2019,PopChangeRate0010,NetMigrationRate0010,NaturalChangeRate0010,Immigration_Rate_2000_2010,...,WhiteVetsPct,BlackVetsPct,HispanicVetsPct,OtherRaceVetsPct,LessThanHSVetsPct,HighSchOnlyVetsPct,SomeCollegeVetsPct,CollegeDegreeVetsPct,EmployeedVetsPct,UnemployeedVetsPct
0,0.317,2.461,4903185,1.059,1.402,0.809,7.48,3.30,3.30,1.222931,...,72.534198,22.945607,2.022065,3.059596,6.396034,28.257189,37.385370,27.961408,71.330519,4.319786
1,0.605,2.001,55869,0.686,1.315,-0.029,24.96,11.87,5.46,-0.010222,...,84.028832,9.673748,5.576631,1.119120,3.007812,20.859375,31.621094,44.511719,86.549340,3.558460
2,2.469,21.911,223234,21.001,0.910,0.714,29.80,26.17,3.32,1.584455,...,90.621980,5.360321,1.439251,2.929360,3.612916,27.659792,36.676731,32.050560,74.927707,3.430532
3,-0.748,-9.664,24686,-8.797,-0.867,0.161,-5.44,-4.80,2.29,1.828365,...,59.563253,37.349398,1.054217,3.087349,12.349398,39.834337,36.746988,11.069277,49.752066,9.966777
4,0.121,-2.081,22394,-2.099,0.017,0.525,10.03,6.43,2.10,0.341485,...,79.134682,18.492673,2.372645,0.000000,6.901510,54.708843,22.286125,16.103523,56.901408,0.000000


In [16]:
# Split into features and target arrays
y = combined_df[y_column].values
X = combined_df.drop([y_column],1).values

In [17]:
# Split preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42)

In [18]:
# Scale the data
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# **************************************
# **************************************
# Neural Network - Sequential
# **************************************
# **************************************
def run_model_sequential():

    # Define the model
    number_input_features = len(X_train_scaled[0])

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=kS_nn_first_units,
                input_dim=number_input_features,
                activation=kS_nn_first_activation))

    # Additional hidden layers
    for layer in kS_nn_deep_layers:
        nn.add(tf.keras.layers.Dense(units=layer['units'],
                                     activation=layer['activation']))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=kS_nn_output_units, activation=kS_nn_output_activation))

    # Check the structure of the model
    print(nn.summary())

    # Compile the model
    nn.compile(loss=kS_nn_compile_loss, optimizer=kS_nn_compile_optimizer, metrics=kS_nn_compile_metrics)

    # Train the model
    fit_model = nn.fit(X_train_scaled, y_train,epochs=kS_nn_train_epochs)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

    nn.save(kS_nn_file)

# end run_model_sequential()
# **************************************


In [20]:
# **************************************
# **************************************
# **************************************
run_model_sequential()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 816       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 1,521
Trainable params: 1,521
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
78/78 [==============================] - 0s 817us/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/50
7